# Lista 7

## scikit i Optuna

(6pkt + 3pkt)

Na liście znajduje się 5 zadań. Zadanie *optuna* jest za 2pkt, a inne zadania za 1pkt. Po rozwiązaniu zadania pokaż kod prowadzącemu i odpowiedz na **pytanie kontrolne** — tylko wtedy przyznajemy punkty. Dodatkowo prześlij zadanie na platformie skos.

Zadania dodatkowe oznaczone są ⭐️. Za wykonanie każdego z nich otrzymasz 1pkt. 

## KFold

Wykonaj walidację krzyżową `KFold` (5 foldów) dla modelu `LinearRegression` na zbiorze California Housing.
Zapisz MSE dla każdego folda i podaj średnie MSE.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

X, y = # Uzupełnij

kf = # Uzupełnij
model = # Uzupełnij

mse_scores = []

for train_idx, test_idx in kf.split(X):
    model.fit(X[train_idx], y[train_idx])
    preds = model.predict(X[test_idx])
    mse_scores.append(mean_squared_error(y[test_idx], preds))

print("MSE fold-by-fold:", mse_scores)
print("Średnie MSE:", np.mean(mse_scores))

MSE fold-by-fold: [0.5558915986952412, 0.5276562547736297, 0.5092832097248743, 0.5048507784142172, 0.5551804780114779]
Średnie MSE: 0.5305724639238881


## StratifiedKFold

Na zbiorze Iris zbuduj problem binarny: „Setosa” vs „inne”.
Porównaj proporcje klas w foldach między `KFold` a `StratifiedKFold`.
Przeprowadź walidację `LogisticRegression` (accuracy).

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

X, y_raw = load_iris(return_X_y=True)
y = (y_raw == 0).astype(int)

kf = # Uzupełnij (KFold)
skf = # Uzupełnij (StratifiedKFold)

print("Udział klasy pozytywnej (KFold):")
for _, test_idx in kf.split(X):
    print(y[test_idx].mean())

print("\nUdział klasy pozytywnej (StratifiedKFold):")
for _, test_idx in skf.split(X, y):
    print(y[test_idx].mean())

model = LogisticRegression(max_iter=200)
scores = # Uzupełnij
print("\nAccuracy:", scores.mean())

Udział klasy pozytywnej (KFold):
0.3333333333333333
0.43333333333333335
0.4
0.26666666666666666
0.23333333333333334

Udział klasy pozytywnej (StratifiedKFold):
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333

Accuracy: 1.0


## GridSearchCV

Wykonaj GridSearchCV dla `SVC`.
Siatka parametrów:

* C: [0.1, 1, 10]
* gamma: ["scale", "auto"]
* kernel: ["rbf", "poly"]

Użyj `StratifiedKFold(5)`. Wyświetl najlepsze parametry i wynik.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

X, y = load_iris(return_X_y=True)

param_grid = # Uzupełnij

grid = # Uzupełnij

grid.fit(X, y)

print("Najlepsze parametry:", # Uzupełnji
print("Najlepszy wynik:", # Uzupełnji

Najlepsze parametry: {'C': 1, 'gamma': 'scale', 'kernel': 'poly'}
Najlepszy wynik: 0.9800000000000001


## RandomizedSearchCV

Wykonaj RandomizedSearchCV dla `RandomForestClassifier`.
Parametry losowane:

* n_estimators: 100–999
* max_depth: 2–39

30 losowań. Wypisz najlepsze parametry i accuracy.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

X, y = load_iris(return_X_y=True)

param_dist = # Uzupełnij

rnd = # Uzupełnij

rnd.fit(X, y)

print("Najlepsze parametry:", # Uzupełnij
print("Najlepszy wynik:", # Uzupełnij

Najlepsze parametry: {'max_depth': 30, 'n_estimators': 370}
Najlepszy wynik: 0.9666666666666668


## Optuna

Twoim zadaniem jest przeprowadzić pełną optymalizację hiperparametrów modelu `GradientBoostingClassifier` z użyciem Optuny.

Wykonaj:

1. Wczytaj dane z `sklearn.datasets`.
2. Zdefiniuj funkcję celu, w której:

   * pobierzesz hiperparametry z `trial.suggest_*`,
   * stworzysz model `GradientBoostingClassifier`,
   * ocenisz go 5-krotną walidacją krzyżową,
   * zwrócisz średnią dokładność.
3. Uruchom optymalizację z `TPESampler` + `MedianPruner`, ustawiając parametry:

   * `direction="maximize"`
   * `storage="sqlite:///gb_optuna.db"`
   * `load_if_exists=True`
4. Uruchom drugą optymalizację z `RandomSampler` (tylko porównanie, bez zapisu do bazy).
5. Wygeneruj kilka (minimum 3) wykresy z `optuna.visualization` na podstawie study zapisanej w SQLite.
6. Uruchom Optuna Dashboard na bazie `gb_optuna.db`.
7. Wydrukuj porównanie wyników TPE vs RandomSampler.

In [ ]:
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.datasets import load_breast_cancer

# ---------------------------------------------------------------
# 1) Wczytanie danych
# ---------------------------------------------------------------

X, y = load_breast_cancer(return_X_y=True)

# ---------------------------------------------------------------
# 2) Funkcja celu używana we wszystkich optymalizacjach
# ---------------------------------------------------------------

def objective(trial):

    params = {
        "n_estimators": # Uzupełnij (int, 50 - 400)
        "learning_rate": # Uzupełnij (float, 0.01 - 0.3)
        "max_depth": # Uzupełnij (int, 1 - 6)
        "subsample": # Uzupełnij (float, 0.5 - 1.0)
        "min_samples_split": # Uzupełnij (int, 2 - 20)
    }

    model = GradientBoostingClassifier(**params)

    cv = # Uzupełnij (StratifiedKFold)
    scores = # Uzupełnij (cross_val_score, scoring="accuracy")

    return scores.mean()

# ---------------------------------------------------------------
# 3) Główna optymalizacja — TPE + zapis do SQLite
# ---------------------------------------------------------------

study_sqlite = optuna.create_study(
    study_name="gb_sqlite_study",
    storage="sqlite:///gb_optuna.db",
    direction="maximize",
    load_if_exists=True,
    sampler= # Uzupełnij 
    pruner= # Uzupełnij
)

# Uzupełnij (study_sqlite.optimize, show_progress_bar=True)

print("Najlepsze parametry TPE:", study_sqlite.best_params)
print("Najlepsze accuracy TPE:", study_sqlite.best_value)

# ---------------------------------------------------------------
# 4) Druga optymalizacja — RandomSampler (bez zapisu do bazy)
# ---------------------------------------------------------------

study_random = # Uzupełnji (create_study, direction="maximize", optuna.samplers.RandomSampler(seed=42))

# Uzupełnij (study_random.optimize)

print("Najlepsze accuracy RandomSampler:", study_random.best_value)

# ---------------------------------------------------------------
# 5) Wizualizacje
# ---------------------------------------------------------------

from optuna.visualization import (
    plot_optimization_history,
    plot_parallel_coordinate,
    plot_slice,
    plot_contour,
    plot_param_importances,
    plot_intermediate_values,
    plot_edf,
    plot_timeline,
    plot_rank,
)

# Uzupełnij

# ---------------------------------------------------------------
# 6) Dashboard — komenda do uruchomienia w terminalu:
# ---------------------------------------------------------------
# optuna-dashboard sqlite:///gb_optuna.db

# ---------------------------------------------------------------
# 7) Porównanie wyników
# ---------------------------------------------------------------

print("===============================================")
print("PORÓWNANIE")
print("TPE (SQLite):   ", study_sqlite.best_value)
print("RandomSampler:   ", study_random.best_value)
print("===============================================")

[I 2025-11-23 19:28:41,956] A new study created in RDB with name: gb_sqlite_study


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-23 19:28:42,626] Trial 0 finished with value: 0.95960254618848 and parameters: {'n_estimators': 181, 'learning_rate': 0.28570714885887566, 'max_depth': 5, 'subsample': 0.7993292420985183, 'min_samples_split': 4}. Best is trial 0 with value: 0.95960254618848.
[I 2025-11-23 19:28:43,216] Trial 1 finished with value: 0.9525850023288308 and parameters: {'n_estimators': 104, 'learning_rate': 0.026844247528777843, 'max_depth': 6, 'subsample': 0.8005575058716043, 'min_samples_split': 15}. Best is trial 0 with value: 0.95960254618848.
[I 2025-11-23 19:28:43,465] Trial 2 finished with value: 0.9666511411271541 and parameters: {'n_estimators': 57, 'learning_rate': 0.29127385712697834, 'max_depth': 5, 'subsample': 0.6061695553391381, 'min_samples_split': 5}. Best is trial 2 with value: 0.9666511411271541.
[I 2025-11-23 19:28:43,956] Trial 3 finished with value: 0.9490607048594939 and parameters: {'n_estimators': 114, 'learning_rate': 0.09823025045826593, 'max_depth': 4, 'subsample': 0.

[I 2025-11-23 19:29:17,825] A new study created in memory with name: no-name-9b81517d-9f45-444e-be54-0de0e2e83d64


[I 2025-11-23 19:29:17,815] Trial 49 finished with value: 0.9543393882937432 and parameters: {'n_estimators': 85, 'learning_rate': 0.13146132324344587, 'max_depth': 6, 'subsample': 0.6946367300734071, 'min_samples_split': 5}. Best is trial 41 with value: 0.9701443875174662.
Najlepsze parametry TPE: {'n_estimators': 379, 'learning_rate': 0.27932969678850167, 'max_depth': 4, 'subsample': 0.5824131016865502, 'min_samples_split': 11}
Najlepsze accuracy TPE: 0.9701443875174662


...

Najlepsze accuracy RandomSampler: 0.9683589504735289


[W 2025-11-23 19:29:47,234] You need to set up the pruning feature to utilize `plot_intermediate_values()`


/var/folders/30/p871csfd59bg0hq53w_t9pw80000gn/T/ipykernel_2807/428174880.py:86: ExperimentalWarning:

plot_timeline is experimental (supported from v3.2.0). The interface can change in the future.



/var/folders/30/p871csfd59bg0hq53w_t9pw80000gn/T/ipykernel_2807/428174880.py:87: ExperimentalWarning:

plot_rank is experimental (supported from v3.2.0). The interface can change in the future.



PORÓWNANIE
TPE (SQLite):    0.9701443875174662
RandomSampler:    0.9683589504735289


## ⭐️ GroupKFold

Wygeneruj zbiór danych składający się z **500 próbek** oraz **10 cech**.
Następnie przydziel każdą próbkę do jednej z **50 grup**, tak aby każda grupa zawierała dokładnie **10 próbek**.

Celem zadania jest sprawdzenie, jak działa walidacja krzyżowa z podziałem na grupy.
Użyj **GroupKFold(5)** – oznacza to, że dane będą dzielone tak, aby **żadne próbki z tej samej grupy nie trafiły jednocześnie do zbioru treningowego i testowego**.

Na tak przygotowanych danych wykonaj walidację krzyżową modelu **RandomForestClassifier**, a następnie wypisz średnią dokładność.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold, cross_val_score
import numpy as np

# 1) Generowanie sztucznych danych
X, y = make_classification(n_samples=500, n_features=10, random_state=42)

# 2) Przydział 500 próbek do 50 grup (po 10 próbek w każdej)
groups = np.repeat(np.arange(50), 10)

# Uzupełnij

Accuracy: 0.9480000000000001


## ⭐️ TimeSeriesSplit

Wygeneruj **sztuczny szereg czasowy** składający się z 1000 obserwacji.
Przyjmij, że cechą wejściową jest indeks czasu (0,1,2,…,999), a wartością docelową sygnał sinusoidalny z zakłóceniem (szumem gaussowskim).

Następnie zastosuj **TimeSeriesSplit(5)**, który wykonuje walidację krzyżową dostosowaną do danych sekwencyjnych — każdy kolejny fold używa coraz większej części historii jako zbioru treningowego.

Na każdym foldzie wytrenuj model **GradientBoostingRegressor**, następnie policz i wypisz **MAE (Mean Absolute Error)** na zbiorze testowym.

Wynik powinien zawierać wartość MAE dla każdego folda osobno.

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

# 1) Generowanie sztucznego szeregu czasowego
np.random.seed(42)
X = np.arange(1000).reshape(-1, 1)
y = np.sin(X[:, 0] / 20) + np.random.normal(scale=0.1, size=1000)

# Uzupełnij

Fold 0: MAE = 0.8288663064508833
Fold 1: MAE = 0.7330599469306064
Fold 2: MAE = 0.6298690477081538
Fold 3: MAE = 0.9575344251525488
Fold 4: MAE = 0.6518714737382351


## ⭐️ Optymalizacja hiperparametrów SVM z użyciem Optuny

Twoim zadaniem jest zoptymalizować hiperparametry modelu **Support Vector Machine (SVC)** z wykorzystaniem biblioteki **Optuna**.

Wykonaj następujące kroki:

1. Wczytaj zbiór danych **Wine** ze `sklearn.datasets`.
2. Podziel dane na zbiór treningowy i testowy (np. 80/20).
3. Zdefiniuj funkcję celu `objective(trial)`, która:

   * wybierze hiperparametry `C` oraz `gamma` (oba w skali logarytmicznej),
   * wybierze kernel spośród: `"rbf"`, `"poly"`, `"sigmoid"`,
   * utworzy model `SVC` z dobranymi parametrami,
   * wykona walidację krzyżową (5-fold),
   * zwróci średnią dokładność walidacyjną jako wartość celu.
4. Utwórz study używając `TPESampler`.
5. Uruchom optymalizację na 40 próbach.
6. Wypisz najlepsze hiperparametry oraz najlepszy uzyskany wynik.
7. Wytrenuj finalny model na pełnym zbiorze treningowym z najlepszymi hiperparametrami i oceń go na zbiorze testowym — wypisz finalny accuracy.

Wyniki powinny pokazać:

* najlepsze hiperparametry wybrane przez Optunę,
* wynik walidacji,
* wynik na zbiorze testowym.

In [ ]:
import optuna
from sklearn.svm import SVC
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

# 1) Wczytanie danych Wine
X, y = load_wine(return_X_y=True)

# 2) Podział danych (80% train, 20% test)
X_train, X_test, y_train, y_test = # Uzupełnij

# 3) Funkcja celu Optuny
def objective(trial):
    # Uzupełnij
    pass

# 4) Utworzenie study
study = # Uzupełnij

# 5) Optymalizacja (40 prób)
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("Najlepsze parametry:", study.best_params)
print("Najlepszy wynik CV:", study.best_value)

# 6) Trening finalnego modelu na najlepszych parametrach
# Uzupełnij

# 7) Ocena na zbiorze testowym
# Uzupełnij

[I 2025-11-23 19:30:18,829] A new study created in memory with name: no-name-e8cbfe0a-f635-4d43-a565-54def3f2a7de


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2025-11-23 19:30:19,147] Trial 0 finished with value: 0.9431034482758621 and parameters: {'C': 2.0688721719741974, 'gamma': 0.08690213727135686, 'kernel': 'poly'}. Best is trial 0 with value: 0.9431034482758621.
[I 2025-11-23 19:30:19,157] Trial 1 finished with value: 0.4012315270935961 and parameters: {'C': 262.0655105908605, 'gamma': 1.8589625277718413, 'kernel': 'rbf'}. Best is trial 0 with value: 0.9431034482758621.
[I 2025-11-23 19:30:19,554] Trial 2 finished with value: 0.9431034482758621 and parameters: {'C': 0.010174730286137303, 'gamma': 0.1536369218785644, 'kernel': 'poly'}. Best is trial 0 with value: 0.9431034482758621.
[I 2025-11-23 19:30:19,565] Trial 3 finished with value: 0.4012315270935961 and parameters: {'C': 0.006979162757999213, 'gamma': 0.00016406329744480543, 'kernel': 'rbf'}. Best is trial 0 with value: 0.9431034482758621.
[I 2025-11-23 19:30:19,572] Trial 4 finished with value: 0.4012315270935961 and parameters: {'C': 703.1897887631378, 'gamma': 0.0319543506